# LIBRARIES

In [168]:
import pandas as pd

import numpy as np

import os

from sklearn.linear_model import LinearRegression

# CONFIGURATION

In [169]:
sOutputSymbol = 'BTCUSD'
sModelType = 'MLP'

dfDesign = pd.read_csv('Data\Design_'+ sOutputSymbol +'_'+ sModelType +'.csv', index_col = 'RunOrder')

In [170]:
iNumberOfIteration = 8
fDelta = 0.1

# STEEPEST DESCENT

In [171]:
dfFactors = dfDesign[['Batch Size', 'Number of Hidden Neurons']]
dfResponse = dfDesign['Response'].to_frame()
iMaximumTrails = dfDesign.index.max()

## Fit Linear Regression Model

In [172]:
oLinearRegression  = LinearRegression()
oLinearRegression.fit(dfFactors, dfResponse)

LinearRegression()

## Coefficient of Determination

In [173]:
oLinearRegression.score(dfFactors, dfResponse)

0.2682917502795865

## Equation

In [181]:
oLinearRegression.coef_

array([[4109.88158092, 6333.47471504]])

In [182]:
oLinearRegression.intercept_

array([454054.96466098])

## Set New Experiments

In [174]:
aRatiosToDecrease = ((oLinearRegression.coef_)/(oLinearRegression.coef_[0][0]))*-1 #first factor is used as base factor
aRatiosToDecrease = aRatiosToDecrease[0] * fDelta

In [175]:
aCenterValues = np.array((dfFactors.max()+dfFactors.min())/2)
aIterations = range(1, iNumberOfIteration + 1)
aNewExperiments = aCenterValues + (np.array(aIterations).reshape(-1, 1) * aRatiosToDecrease * aCenterValues)
aNewExperiments = aNewExperiments.astype(np.int)

In [176]:
dfNewExperiments =pd.DataFrame(
    data = aNewExperiments,
    columns = dfFactors.columns,
    index = iMaximumTrails + aIterations
)
dfNewExperiments

,Batch Size,Number of Hidden Neurons
31,129,121
32,115,99
33,100,77
34,86,55
35,72,33
36,57,10
37,43,-11
38,28,-33


## Drop Experiments Less Than 10

In [177]:
ixExperimetnsLessThan10 = dfNewExperiments[(dfNewExperiments < 10).any(axis= 1)].index
dfNewExperiments.loc[ixExperimetnsLessThan10]

,Batch Size,Number of Hidden Neurons
37,43,-11
38,28,-33


In [178]:
dfNewExperiments.drop(ixExperimetnsLessThan10, axis = 0 , inplace = True)

## Save New Experiments

In [179]:
dfNewExperiments.to_csv('Data\Steepest Descent_'+ sOutputSymbol +'_'+ sModelType +'.csv')

In [180]:
dfNewExperiments

,Batch Size,Number of Hidden Neurons
31,129,121
32,115,99
33,100,77
34,86,55
35,72,33
36,57,10
